Capstone project week3 part 3 of 3: Segmenting and Clustering Neighborhoods of Toronto 

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# !pip instead of conda #forum saying using this one instead

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('library imported')

library imported


## 1. making the dataframe from previous assignment of Toronto neighborhood 


In [26]:
Toronto_Neib=pd.read_csv('Toronto_Neib_Coor.csv',index_col=0)
Toronto_Neib.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


In [35]:
print('there are {} borough and {} neighborhood'.format(len(Toronto_Neib['Borough'].unique()),Toronto_Neib.shape[0]))

there are 10 borough and 103 neighborhood


In [55]:
Toronto_Neib_onlyToronto=Toronto_Neib[Toronto_Neib['Borough'].str.contains('Toronto')]
Toronto_Neib_onlyToronto.reset_index(drop=True,inplace=True)
Toronto_Neib_onlyToronto.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


#### use geopy library t oget teh lat/long of Toronto

In [40]:
address = 'Toronto, On'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### create Toronto map with neiborhood superimpoosed on it


In [51]:
#create map
map_Toronto=folium.Map(location=(latitude,longitude),zoom_start=10)
for lat, lng, borough, neighborhood in zip(Toronto_Neib_onlyToronto['Latitude'], Toronto_Neib_onlyToronto['Longitude'], Toronto_Neib_onlyToronto['Borough'], Toronto_Neib_onlyToronto['Neighborhood']):
    label = '{}, {}'.format(Toronto_Neib_onlyToronto, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Define Foursquare Credentials and Version

In [56]:
CLIENT_ID = 'V02UOPVOXVPB3YMTOU0I0X3VHIA0QJJ3Q5XXRUQPCBYJ4UMK' # your Foursquare ID
CLIENT_SECRET = 'G4WMZYWQVBZL4KDO51THA5V0GSEQWY2UNYRTU4MNFJFOU3CL' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V02UOPVOXVPB3YMTOU0I0X3VHIA0QJJ3Q5XXRUQPCBYJ4UMK
CLIENT_SECRET:G4WMZYWQVBZL4KDO51THA5V0GSEQWY2UNYRTU4MNFJFOU3CL


In [57]:
#check the first neiborhood to see what venue look like first
Toronto_Neib_onlyToronto.loc[0,'Neighborhood']

'Regent Park , Harbourfront'

In [67]:
#get lat/long of this neighborhood
neighborhood_latitude=Toronto_Neib_onlyToronto.loc[0,'Latitude']
neighborhood_longitude=Toronto_Neib_onlyToronto.loc[0,'Longitude']
neighborhood_name=Toronto_Neib_onlyToronto.loc[0,'Neighborhood']

print('lat/log {}{} with neighnorhood name as of {}.'.format(neighborhood_latitude,neighborhood_longitude,neighborhood_name))


lat/log 43.6542599-79.3606359 with neighnorhood name as of Regent Park , Harbourfront.


#### Now, let's get the top 20 venues that are in Marble Hill within a radius of 500 meters.

In [68]:
# type your answer here
LIMIT = 20
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=V02UOPVOXVPB3YMTOU0I0X3VHIA0QJJ3Q5XXRUQPCBYJ4UMK&client_secret=G4WMZYWQVBZL4KDO51THA5V0GSEQWY2UNYRTU4MNFJFOU3CL&v=20180604&ll=43.6542599,-79.3606359&radius=500&limit=20'

In [69]:
#get the result and check it
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9e0e4d949393001bf6ce37'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [70]:
# save the search as json file so as to open in jason editor, which is easy to look at
with open('Toronto-venue-examle.json', 'w') as json_file:
     json.dump(results, json_file)

In [72]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [73]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [74]:
nearby_venues.shape

(20, 4)

## 2.  expolring Toronto neighborhood 


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [75]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.

In [76]:
Toronto_venues = getNearbyVenues(names=Toronto_Neib_onlyToronto['Neighborhood'],
                                   latitudes=Toronto_Neib_onlyToronto['Latitude'],
                                   longitudes=Toronto_Neib_onlyToronto['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Roseda

In [77]:
Toronto_venues.shape

(639, 7)

In [79]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [78]:
#check how many venues each neighnorhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"Brockton , Parkdale Village , Exhibition Place",20,20,20,20,20,20
Business reply mail Processing CentrE,17,17,17,17,17,17
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",15,15,15,15,15,15
Central Bay Street,20,20,20,20,20,20
Christie,19,19,19,19,19,19
Church and Wellesley,20,20,20,20,20,20
"Commerce Court , Victoria Hotel",20,20,20,20,20,20
Davisville,20,20,20,20,20,20


In [80]:
print('there are {} unique categories'.format(len(Toronto_venues['Venue Category'].unique())))

there are 166 unique categories


## 3. Analyze Each Neighborhood

In [81]:
#so we have 639 venues but 166 unique categories

# one hot encoding
Toronto_venues_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_venues_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_venues_onehot.columns[-1]] + list(Toronto_venues_onehot.columns[:-1])
Toronto_venues_onehot = Toronto_venues_onehot[fixed_columns]

Toronto_venues_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park , Har

In [82]:
Toronto_venues_onehot.shape

(639, 166)

In [85]:
#group rows by neighborhood and taking the mean of frequency of occurence of each category
Toronto_venues_onehot_grouped = Toronto_venues_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_venues_onehot_grouped.head()


,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.0,0.000000,0.05,0.1,0.0,0.0,0.05,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.05,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.0,0.050000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.050000,0.0,0.0,0.00,0.000000,0.05,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.0,0.050000,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.1,0.000000,0.00,0.0,0.050000,0.0,0.0,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.10,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.05,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.05,0.0,0.0,0.000000,0.05,0.05,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.050000,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.

In [93]:
Toronto_venues_onehot_grouped.shape

(39, 166)

In [104]:
#Let's print each neighborhood along with the top 5 most common venue
num_top_venues = 5

for hood in Toronto_venues_onehot_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_venues_onehot_grouped[Toronto_venues_onehot_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0    Cocktail Bar  0.10
1  Farmers Market  0.10
2        Beer Bar  0.10
3          Bakery  0.05
4     Coffee Shop  0.05


----Brockton , Parkdale Village , Exhibition Place----
                venue  freq
0                Café  0.15
1         Coffee Shop  0.10
2      Breakfast Spot  0.10
3       Grocery Store  0.05
4  Italian Restaurant  0.05


----Business reply mail Processing CentrE----
                  venue  freq
0           Yoga Studio  0.06
1                   Spa  0.06
2         Garden Center  0.06
3                Garden  0.06
4  Fast Food Restaurant  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.20
1    Airport Lounge  0.13
2     Boat or Ferry  0.07
3          Boutique  0.07
4  Sculpture Garden  0.07


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.30
1    

#### Let's put that into a *pandas* dataframe

In [105]:
#define a function t osort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [107]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_venues_onehot_grouped['Neighborhood']

for ind in np.arange(Toronto_venues_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_venues_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Beer Bar,Cocktail Bar,Farmers Market,Concert Hall,Seafood Restaurant,Museum,Basketball Stadium,Bakery,Restaurant,Liquor Store
1,"Brockton , Parkdale Village , Exhibition Place",Café,Coffee Shop,Breakfast Spot,Italian Restaurant,Stadium,Office,Performing Arts Venue,Pet Store,Burrito Place,Restaurant
2,Business reply mail Processing CentrE,Yoga Studio,Spa,Gym / Fitness Center,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Garden Center,Park,Pizza Place
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Boutique,Airport Terminal,Harbor / Marina,Sculpture Garden,Plane,Boat or Ferry,Airport Gate,Airport Food Court
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Bar,Steakhouse,Ramen Restaurant,Spa,Sushi Restaurant,Park,Japanese Restaurant


In [108]:
neighborhoods_venues_sorted.shape

(39, 11)

## 4. Cluster Neighborhoods

In [109]:
Toronto_venues_onehot_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Eastern European Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skate Park,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.0,0.000000,0.05,0.1,0.0,0.0,0.05,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.05,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.05,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.0,0.050000,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.050000,0.0,0.0,0.00,0.000000,0.05,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.05,0.0,0.0
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.05,0.0,0.050000,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.1,0.000000,0.00,0.0,0.050000,0.0,0.0,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.10,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.05,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.05,0.0,0.0,0.000000,0.05,0.05,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.050000,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.05,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.

In [117]:
#run k-means to do the clustering into 5 clusters
# set number of clusters
kclusters = 5

Toronto_venues_onehot_grouped_clustering = Toronto_venues_onehot_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_venues_onehot_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 4, 4, 1, 4, 1, 1])

In [120]:
Toronto_Neib_onlyToronto.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [121]:
#create new datafram that includes the cluster as well as top 1o venues in the neighborhood
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_Neib_onlyToronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal_code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,4,Coffee Shop,Breakfast Spot,Historic Site,Gym / Fitness Center,Restaurant,Distribution Center,Chocolate Shop,Spa,Pub,Bakery
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Hobby Shop,Park,Mexican Restaurant,Creperie,Burrito Place,Burger Joint,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Coffee Shop,Comic Shop,Sandwich Place,Ramen Restaurant,Electronics Store,Plaza,Sporting Goods Shop,Steakhouse,Burrito Place
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Gastropub,Coffee Shop,Hotel,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Hostel,BBQ Joint,Creperie,Gym
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Trail,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant


In [123]:
#finally time to visualize
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [124]:
#cluster 1
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,0,Park,Swim School,Bus Line,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant
33,Downtown Toronto,0,Park,Trail,Playground,Wine Bar,Creperie,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


In [125]:
#cluster 2
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Hobby Shop,Park,Mexican Restaurant,Creperie,Burrito Place,Burger Joint,Diner
4,East Toronto,1,Health Food Store,Pub,Trail,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Cuban Restaurant
5,Downtown Toronto,1,Beer Bar,Cocktail Bar,Farmers Market,Concert Hall,Seafood Restaurant,Museum,Basketball Stadium,Bakery,Restaurant,Liquor Store
8,Downtown Toronto,1,Seafood Restaurant,Café,Sushi Restaurant,Hotel,Plaza,Bar,Pizza Place,Speakeasy,Gym / Fitness Center,Concert Hall
9,West Toronto,1,Pharmacy,Bakery,Supermarket,Brewery,Middle Eastern Restaurant,Music Venue,Park,Grocery Store,Gym / Fitness Center,Bar
10,Downtown Toronto,1,Café,Park,Plaza,Dessert Shop,Japanese Restaurant,Salad Place,Deli / Bodega,Hotel,Skating Rink,Sporting Goods Shop
11,West Toronto,1,Bar,Wine Bar,Record Shop,Art Gallery,Asian Restaurant,Beer Store,Brewery,Cocktail Bar,Cuban Restaurant,French Restaurant
12,East Toronto,1,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place,Dessert Shop,Restaurant,Brewery,Bubble Tea Shop,Pub
15,East Toronto,1,Fast Food Restaurant,Sandwich Place,Movie Theater,Pizza Place,Burrito Place,Pet Store,Steakhouse,Restaurant,Gym,Sushi Restaurant
17,East Toronto,1,Coffee Shop,Café,Fish Market,Middle Eastern Restaurant,Comfort Food Restaurant,Pet Store,Brewery,Bookstore,Sandwich Place,Italian Restaurant


In [126]:
#cluster 3
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,2,Playground,Summer Camp,Wine Bar,Creperie,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


In [127]:
#cluster 4
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,3,Garden,Home Service,Wine Bar,Cuban Restaurant,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega


In [128]:
#cluster 5
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,4,Coffee Shop,Breakfast Spot,Historic Site,Gym / Fitness Center,Restaurant,Distribution Center,Chocolate Shop,Spa,Pub,Bakery
2,Downtown Toronto,4,Café,Coffee Shop,Comic Shop,Sandwich Place,Ramen Restaurant,Electronics Store,Plaza,Sporting Goods Shop,Steakhouse,Burrito Place
3,Downtown Toronto,4,Gastropub,Coffee Shop,Hotel,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Hostel,BBQ Joint,Creperie,Gym
6,Downtown Toronto,4,Coffee Shop,Italian Restaurant,Sandwich Place,Bar,Steakhouse,Ramen Restaurant,Spa,Sushi Restaurant,Park,Japanese Restaurant
7,Downtown Toronto,4,Grocery Store,Café,Coffee Shop,Park,Gas Station,Athletics & Sports,Candy Store,Restaurant,Diner,Italian Restaurant
13,Downtown Toronto,4,Coffee Shop,Café,Gym,Hotel,Pub,Beer Bar,Bakery,Deli / Bodega,Steakhouse,Restaurant
14,West Toronto,4,Café,Coffee Shop,Breakfast Spot,Italian Restaurant,Stadium,Office,Performing Arts Venue,Pet Store,Burrito Place,Restaurant
16,Downtown Toronto,4,Café,Gastropub,Coffee Shop,Restaurant,Gym,Deli / Bodega,Pub,Beer Bar,Bakery,Gym / Fitness Center
24,Central Toronto,4,Café,Coffee Shop,Sandwich Place,Indian Restaurant,Liquor Store,Pub,History Museum,Burger Joint,Cosmetics Shop,BBQ Joint
31,Central Toronto,4,Pub,Coffee Shop,Light Rail Station,Liquor Store,Bank,Bagel Shop,Sports Bar,Pizza Place,Supermarket,Sushi Restaurant
